In [1]:
# ==============================================================================
# SECTION 4: MODEL FINE-TUNING SCRIPT
# ==============================================================================
# ⚠️  คำเตือน: เซลล์นี้ต้องรันบน Environment ที่มี GPU เท่านั้น
# ------------------------------------------------------------------------------

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from trl import SFTTrainer

# --- เตรียมข้อมูลตัวอย่างสำหรับเทรน (เพื่อให้โค้ดรันได้) ---
training_examples = [
    {"text": "Thought: (โอเค มีปัญหาเกิดขึ้น ตั้งสติ แล้วแก้ไขไปทีละขั้นตอน)"},
    {"text": "Dialogue: ต้องขออภัยในความไม่สะดวกด้วยจริงๆ นะคะ เรื่องรสชาติไอศกรีมค่ะ เข้าใจเลยค่ะ เดี๋ยวทางเราจะรีบ เปลี่ยนถ้วยใหม่ ให้นะคะ!"}
]
train_dataset = Dataset.from_list(training_examples)


# --- ตั้งค่าการโหลดโมเดล 4-bit ---
model_name = "microsoft/Phi-3-mini-4k-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# --- โหลด Base Model และ Tokenizer ---
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# --- ตั้งค่า LoRA ---
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

# --- ตั้งค่า Training Arguments ---
training_args = TrainingArguments(
    output_dir="./hr-agent-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,
    save_strategy="epoch",
)

# --- สร้าง Trainer และเริ่มการเทรน ---
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
)

# trainer.train() # <--- บรรทัดนี้ให้ใส่ # ไว้ก่อน เพื่อยังไม่ต้องรันจริง
print("✅ Fine-tuning script is ready.")

c:\Users\G2\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


ImportError: cannot import name 'Cache' from 'transformers' (c:\Users\G2\anaconda3\Lib\site-packages\transformers\__init__.py)